In [ ]:
# @title
from IPython.display import display, HTML

display(HTML("""
<script>
const firstCell = document.querySelector('.cell.code_cell');
if (firstCell) {
  firstCell.querySelector('.input').style.pointerEvents = 'none';
  firstCell.querySelector('.input').style.opacity = '0.5';
}
</script>
"""))

html = """
<div style="display:flex; flex-direction:column; align-items:center; text-align:center; gap:12px; padding:8px;">
  <h1 style="margin:0;">👋 Welcome to <span style="color:#1E88E5;">Algopath Coding Academy</span>!</h1>

  <img src="https://raw.githubusercontent.com/sshariqali/mnist_pretrained_model/main/algopath_logo.jpg"
       alt="Algopath Coding Academy Logo"
       width="400"
       style="border-radius:15px; box-shadow:0 4px 12px rgba(0,0,0,0.2); max-width:100%; height:auto;" />

  <p style="font-size:16px; margin:0;">
    <em>Empowering young minds to think creatively, code intelligently, and build the future with AI.</em>
  </p>
</div>
"""

display(HTML(html))

## Day 8 - Part 7: Scaling Up to a Real GPT! 🚀

---

### **The Complete Journey**

**Day 7:**
- Part 1-2: Bigram Language Model (simple baseline)
- Part 3: Understanding Attention (Q, K, V concept)

**Day 8:**
- Part 3: Single-Head Attention
- Part 4: Multi-Head Attention
- Part 5: Complete Transformer Block (+ FFN)
- Part 6: Add & Norm (LayerNorm + Residuals)
- **Part 7: SCALING UP!** ← 📍 You are here!

---

### **Agenda for this Notebook**

| Section | Topic | Description |
|:-------:|-------|-------------|
| 1 | **Setup** | Data pipeline (unchanged) |
| 2 | **🆕 Scaled Hyperparameters** | Real GPT-like settings |
| 3 | **Components with Dropout** | Regularization for large models |
| 4 | **Stacked Blocks** | Multiple Transformer blocks |
| 5 | **Pre-trained Model** | Load saved weights |
| 6 | **Generation** | Generate real Shakespeare! |

---

### **What We're Scaling**

| Parameter | Before | Now | Real GPT-3 |
|-----------|--------|-----|------------|
| `n_embed` | 32 | **384** | 12,288 |
| `block_size` | 8 | **256** | 2,048 |
| `n_head` | 4 | **6** | 96 |
| `n_layer` | 1 | **6** | 96 |
| `dropout` | 0.0 | **0.2** | 0.1 |

**New Additions:**
- **Dropout**: Randomly zeros activations during training (prevents overfitting)
- **Multiple Blocks**: Stack 6 Transformer blocks for deeper processing
- **Final LayerNorm**: Normalize before the final prediction head

This is a **real language model** that can generate coherent Shakespeare! 🎭

---
## Section 1: Setup

In [1]:
import torch

---
## Section 2: Scaled Hyperparameters

**Notice the dramatically different values!**

| Parameter | Old Value | New Value | Why? |
|-----------|-----------|-----------|------|
| `block_size` | 8 | **256** | Longer context = better understanding |
| `n_embed` | 32 | **384** | More dimensions = richer representations |
| `n_head` | 4 | **6** | More heads = more attention patterns |
| `n_layer` | 1 | **6** | More blocks = deeper processing |
| `dropout` | 0.0 | **0.2** | Regularization for large models |
| `learning_rate` | 1e-3 | **3e-4** | Smaller LR for stable training |

In [2]:
batch_size = 32 # how many independent sequences to process in parallel
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embed = 384
n_head = 6
n_layer = 6
dropout = 0.2

---
## Section 3: Data Pipeline (Same as Before)

In [3]:
# Load the tiny shakespeare dataset
dataset = "tiny_shakespeare.txt"

# Load the dataset into a string
with open(dataset, "r", encoding="utf-8") as f:
    text = f.read()

chars = sorted(list(set(text)))
vocab_size = len(chars)

Tokenizer:

In [4]:
stoi = { ch: i for i, ch in enumerate(chars)}
itos = { i: ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

Data split:

In [5]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(len(data) * 0.9)
train_data = data[:n]
val_data = data[n:]

Batch loading:

In [6]:
torch.manual_seed(1337)

def get_batch(split):
    # generate a small batch of input-target pairs
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i+block_size] for i in ix])
    y = torch.stack([data[i+1 : i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

Loss estimation:

In [7]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

---
## Section 4: Model Components with Dropout

**🆕 Dropout Added Throughout!**

Dropout randomly "drops" (zeros out) neurons during training:
- Prevents the model from relying on any single neuron
- Acts as regularization, reducing overfitting
- Only active during training (disabled at inference)

**Where we add dropout:**
1. After attention weights (before multiplying with V)
2. After the projection in Multi-Head Attention
3. After the FFN

```python
self.dropout = torch.nn.Dropout(dropout)  # dropout = 0.2 means 20% dropped
```

In [8]:
class Head(torch.nn.Module):

    def __init__(self, head_size):
        super().__init__()
        self.key = torch.nn.Linear(n_embed, head_size, bias=False)
        self.query = torch.nn.Linear(n_embed, head_size, bias=False)
        self.value = torch.nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)
        wei = q @ k.transpose(-2, -1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = torch.nn.functional.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        out = wei @ v
        return out

Multi-Head Attention (with dropout):

In [9]:
class MultiHeadAttention(torch.nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = torch.nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = torch.nn.Linear(n_embed, n_embed)
        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

Feed-Forward Network (with dropout):

In [10]:
class FeedForward(torch.nn.Module):

    def __init__(self, n_embed):
        super().__init__()
        self.net = torch.nn.Sequential(
            torch.nn.Linear(n_embed, n_embed * 4),
            torch.nn.ReLU(),
            torch.nn.Linear(n_embed * 4, n_embed),
            torch.nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

Complete Block (same as Part 6, but using components with dropout):

In [11]:
class Block(torch.nn.Module):

    def __init__(self, n_embed, n_head):
        super().__init__()
        head_size = n_embed // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embed)
        self.ln1 = torch.nn.LayerNorm(n_embed)
        self.ln2 = torch.nn.LayerNorm(n_embed)

    def forward(self, x):
        x = x + self.sa(self.ln1(x)) # residual connection
        x = x + self.ffwd(self.ln2(x)) # residual connection
        return x

---
## Section 5: Complete Transformer Model

**Key changes for scaling:**

1. **Multiple Blocks**: Instead of 1 block, we stack `n_layer = 6` blocks!
```python
self.blocks = nn.Sequential(*[Block(...) for _ in range(n_layer)])
```

2. **Final LayerNorm**: Before the LM head, one more normalization
```python
self.ln_f = nn.LayerNorm(n_embed)
```

**Architecture:**
**IMAGE HERE**

This is essentially a **mini-GPT**! Same architecture, just smaller scale.

In [ ]:
torch.manual_seed(1337)

class Transformer(torch.nn.Module):

    def __init__(self):
        super().__init__()
        # Each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = torch.nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = torch.nn.Embedding(block_size, n_embed)
        self.blocks = torch.nn.Sequential(*[Block(n_embed, n_head = n_head) for _ in range(n_layer)])
        self.ln_f = torch.nn.LayerNorm(n_embed)
        self.lm_head = torch.nn.Linear(n_embed, vocab_size)

    def forward(self, idx, y = None):
        # idx is (B, T) tensor of indices.
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx) # (B, T, n_embed)
        pos_emb = self.position_embedding_table(torch.arange(T, device = device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x) # (B, T, vocab_size)

        # Loss
        if y is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            y = y.view(B*T)
            loss = torch.functional.F.cross_entropy(logits, y)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) tensor of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = torch.nn.functional.softmax(logits, dim = -1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples = 1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim = 1) # (B, T+1)
        return idx

In [ ]:
model = Transformer()
model = model.to(device)
model.load_state_dict(torch.load('model_weights.pth', map_location=device))
model.eval()

BigramLanguageModel(
  (token_embedding_table): Embedding(65, 384)
  (position_embedding_table): Embedding(256, 384)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-5): 6 x Head(
            (key): Linear(in_features=384, out_features=64, bias=False)
            (query): Linear(in_features=384, out_features=64, bias=False)
            (value): Linear(in_features=384, out_features=64, bias=False)
            (dropout): Dropout(p=0.2, inplace=False)
          )
        )
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (ffwd): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=384, out_features=1536, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1536, out_features=384, bias=True)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
      (ln1): LayerNorm((384,), eps=1e-05, elementwise_aff

---
## Section 6: Loading Pre-trained Weights

Training this model from scratch takes a while (~15-30 minutes on GPU). 

We've pre-trained the model and saved the weights. Let's load them!

---
## Section 7: Training (Optional)

**⚠️ Training is commented out** because we loaded pre-trained weights.

If you want to train from scratch:
1. Comment out the `model.load_state_dict(...)` line above
2. Uncomment the training loop below
3. Run for ~5000 iterations (takes 15-30 min on GPU)

**Training Tips:**
- Lower learning rate (3e-4) is crucial for stable training
- Watch for val_loss to decrease along with train_loss
- If val_loss increases while train_loss decreases → overfitting!

In [14]:
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

In [18]:
# for i in range(max_iters):

#     if i % eval_interval == 0:
#         losses = estimate_loss()
#         print(f"step {i}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

#     # sample a batch of data
#     xb, yb = get_batch('train')

#     # evaluate the loss
#     logits, loss = model(xb, yb)

#     # perform backpropagation
#     loss.backward()

#     # update the weights
#     optimizer.step()

#     # zero the gradients
#     optimizer.zero_grad(set_to_none = True)

step 0: train loss 4.2846, val loss 4.2820
step 500: train loss 2.1101, val loss 2.1662
step 1000: train loss 1.7105, val loss 1.8663
step 1500: train loss 1.5289, val loss 1.7154
step 2000: train loss 1.4320, val loss 1.6319
step 2500: train loss 1.3596, val loss 1.5877
step 3000: train loss 1.3164, val loss 1.5455
step 3500: train loss 1.2737, val loss 1.5340
step 4000: train loss 1.2385, val loss 1.5005
step 4500: train loss 1.2085, val loss 1.4999


Save model weights (for next time):

In [23]:
# Save the model's weights

# torch.save(model.state_dict(), 'model_weights.pth')

---
## Section 8: Text Generation - The Finale!

**The moment of truth!** Let's generate 1000 characters of Shakespeare-like text.

Our model has learned:
- Character patterns and spelling
- Word boundaries and common words
- Simple grammatical structures
- Even some character names and dialogue patterns!

In [15]:
context = torch.zeros((1, 1), dtype = torch.long).to(device) # Since idx 0 is a new line character
out = model.generate(context, max_new_tokens = 1000)
print(decode(out[0].tolist()))



Nurse:
Seeing with that we should some be done,
I'll not deliver such backed with her back:
As way, away, what fall out of my noble mood,
And bloody well make my father hand late:
Give me thee them them on that we love down upon,
If courage preventive her years in she rooten
Against my life to that more would do piece,
So thought that ever now poor great to
The womb cruptiffs are great live, I may post
To fight do I of.

HENRY BOLINGBROKE:
Saddle men, gentle mine mischarate
His honourable carelering nature,
By though thy beam of a cells of permised:
And madam, thou shadows thy witchford, my countryman!

DUKE OF YORK:
My fortune kiss's weapons in my stead?
Canst thou shalt entreated: therefore, fear those thought
May sucking to low her eye out as any barb'd;
Which did cowards she for my lamb despire
By the right creature, helpess upon the name,
That teyrs of his father's nose fellows tongues.

GREEN:
Be that will say this be be thou shalt companied
By this imposition, for God, know he

---
## 🎉 Congratulations! You've Built a GPT!

### 📊 What You've Accomplished

Over Day 7 and Day 8, you've built a complete **GPT-style language model** from scratch!

**Architecture Summary:**
```
┌─────────────────────────────────────────────────────────────────┐
│                         TRANSFORMER                              │
├─────────────────────────────────────────────────────────────────┤
│  Input Token IDs (B, T)                                          │
│       ↓                                                          │
│  Token Embedding (vocab_size → n_embed)                          │
│       +                                                          │
│  Position Embedding (block_size → n_embed)                       │
│       ↓                                                          │
│  ┌───────────────────────────────────────┐                       │
│  │         TRANSFORMER BLOCK × 6          │                       │
│  │  ┌─────────────────────────────────┐  │                       │
│  │  │ LayerNorm → Multi-Head Attention │  │                       │
│  │  │         (+ Residual)             │  │                       │
│  │  ├─────────────────────────────────┤  │                       │
│  │  │ LayerNorm → Feed-Forward Network │  │                       │
│  │  │         (+ Residual)             │  │                       │
│  │  └─────────────────────────────────┘  │                       │
│  └───────────────────────────────────────┘                       │
│       ↓                                                          │
│  Final LayerNorm                                                 │
│       ↓                                                          │
│  Language Model Head (n_embed → vocab_size)                      │
│       ↓                                                          │
│  Output Logits (B, T, vocab_size)                                │
└─────────────────────────────────────────────────────────────────┘
```

### 🔑 Key Concepts Mastered

| Concept | What You Learned |
|---------|-----------------|
| **Tokenization** | Convert text ↔ numbers |
| **Embeddings** | Dense vector representations |
| **Self-Attention** | Tokens communicate via Q-K-V |
| **Multi-Head** | Multiple attention patterns in parallel |
| **Feed-Forward** | Per-token non-linear processing |
| **Layer Norm** | Stabilize activations |
| **Residuals** | Enable deep networks |
| **Dropout** | Prevent overfitting |
| **Causal Masking** | No peeking at the future! |

### 🚀 From Here to ChatGPT

Your model vs. GPT-4:

| Aspect | Your Model | GPT-4 |
|--------|-----------|-------|
| Parameters | ~10 million | ~1.8 trillion |
| Context | 256 tokens | 128,000 tokens |
| Training Data | 1MB Shakespeare | Trillions of tokens |
| Training Time | 30 minutes | Months on thousands of GPUs |
| Architecture | **IDENTICAL!** | **IDENTICAL!** |

**The fundamental architecture is the SAME!** GPT-4 just has:
- More layers, more heads, more dimensions
- Better training data and techniques
- RLHF (Reinforcement Learning from Human Feedback)
- Instruction tuning

### 🎓 What's Next?

Now that you understand Transformers, explore:
- **BERT**: Bidirectional (sees full context, not just past)
- **Vision Transformers (ViT)**: Attention for images
- **Multimodal Models**: Combining text, images, audio
- **Fine-tuning**: Adapting pre-trained models to your tasks
- **RLHF**: Making models helpful and safe

---

## 🙏 Thank You!

You've completed the Day 7-8 journey into the heart of modern AI!

**Remember:** Every ChatGPT, Claude, Gemini, and LLaMA is built on these exact same principles you just learned. You now understand the architecture that's transforming the world! 🌍